## Setting Up:

In [ ]:
import sys, os
sys.path.append(os.path.join(os.getcwd(), '../../')) # Add root of repo to import MBM

import pandas as pd
import warnings
import re
import matplotlib.pyplot as plt
import seaborn as sns
from cmcrameri import cm
import xarray as xr
import massbalancemachine as mbm
from collections import defaultdict
import logging
import joypy
from skorch.callbacks import EarlyStopping, LRScheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset
import torch.nn as nn
from skorch.helper import SliceDataset
from pandas.api.types import CategoricalDtype
from matplotlib.patches import Patch

import matplotlib.pyplot as plt
import geopandas as gpd

from scripts.helpers import *
from scripts.glamos_preprocess import *
from scripts.plots import *
from scripts.config_CH import *
from scripts.xgb_helpers import *
from scripts.geodata import *
from scripts.nn_helpers import *
from scripts.geodata_plots import *
from scripts.NN_networks import *

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

cfg = mbm.SwitzerlandConfig()

In [ ]:
seed_all(cfg.seed)
free_up_cuda()

# Plot styles:
path_style_sheet = 'scripts/example.mplstyle'
plt.style.use(path_style_sheet)
colors = get_cmap_hex(cm.batlow, 10)
color_dark_blue = colors[0]
color_pink = '#c51b7d'

# Glacier outlines:
glacier_outline_sgi = gpd.read_file(
    os.path.join(cfg.dataPath, path_SGI_topo, 'inventory_sgi2016_r2020',
                 'SGI_2016_glaciers_copy.shp'))  # Load the shapefile
glacier_outline_rgi = gpd.read_file(cfg.dataPath + path_rgi_outlines)

test_glaciers = [
    'tortin', 'plattalva', 'sanktanna', 'schwarzberg', 'hohlaub', 'pizol',
    'corvatsch', 'tsanfleuron', 'forno'
]

vois_climate = [
    't2m', 'tp', 'slhf', 'sshf', 'ssrd', 'fal', 'str', 'u10', 'v10'
]

vois_topographical = [
    "aspect_sgi",
    "slope_sgi",
    "hugonnet_dhdt",
    "consensus_ice_thickness",
    "millan_v",
]

## Read GL data:

In [ ]:
data_glamos = getStakesData(cfg)

print('-------------------')
print('Number of glaciers:', len(data_glamos['GLACIER'].unique()))
print('Number of winter and annual samples:', len(data_glamos))
print('Number of annual samples:',
      len(data_glamos[data_glamos.PERIOD == 'annual']))
print('Number of winter samples:',
      len(data_glamos[data_glamos.PERIOD == 'winter']))

In [ ]:
# get number of measurements per glacier:
glacier_info = data_glamos.groupby('GLACIER').size().sort_values(
    ascending=False).reset_index()
glacier_info.rename(columns={0: 'Nb. measurements'}, inplace=True)
glacier_info.set_index('GLACIER', inplace=True)

glacier_loc = data_glamos.groupby('GLACIER')[['POINT_LAT', 'POINT_LON']].mean()

glacier_info = glacier_loc.merge(glacier_info, on='GLACIER')

glacier_period = data_glamos.groupby(['GLACIER', 'PERIOD'
                                      ]).size().unstack().fillna(0).astype(int)

glacier_info = glacier_info.merge(glacier_period, on='GLACIER')

glacier_info['Train/Test glacier'] = glacier_info.apply(
    lambda x: 'Test' if x.name in test_glaciers else 'Train', axis=1)
glacier_info.head(2)

### Assign glaciers to river basin names:

In [ ]:
# === Load RGI glacier IDs ===
rgi_df = pd.read_csv(cfg.dataPath + path_glacier_ids)
rgi_df.columns = rgi_df.columns.str.strip()
rgi_df = rgi_df.sort_values(by='short_name').set_index('short_name')

# === Load SGI region geometries ===
SGI_regions = gpd.read_file(
    os.path.join(cfg.dataPath, path_SGI_topo, 'inventory_sgi2016_r2020',
                 'sgi_regions.geojson'))

# Clean object columns
SGI_regions[SGI_regions.select_dtypes(include='object').columns] = \
    SGI_regions.select_dtypes(include='object').apply(lambda col: col.str.strip())

SGI_regions = SGI_regions.drop_duplicates().dropna()
SGI_regions = SGI_regions.set_index('pk_sgi_region')

# === Map to Level 0 river basins ===
catchment_lv0 = {
    'A': 'Rhine',
    'B': 'Rhone',
    'C': 'Po',
    'D': 'Adige',
    'E': 'Danube'
}
rgi_df['rvr_lv0'] = rgi_df['sgi-id'].str[0].map(catchment_lv0)


# === Map to Level 1 river basins using SGI regions ===
def get_river_basin(sgi_id):
    key = sgi_id.split('-')[0]
    if key not in SGI_regions.index:
        return None
    basin = SGI_regions.loc[key, 'river_basin_name']
    if isinstance(basin, pd.Series):
        return basin.dropna().unique()[0] if not basin.dropna().empty else None
    return basin if pd.notna(basin) else None


rgi_df['rvr_lv1'] = rgi_df['sgi-id'].apply(get_river_basin)

# Final formatting
rgi_df = rgi_df.reset_index().rename(columns={
    'short_name': 'GLACIER'
}).set_index('GLACIER')

glacier_info = glacier_info.merge(rgi_df[['rvr_lv0', 'rvr_lv1']],
                                  on='GLACIER',
                                  how='left')
glacier_info.head()

### Read input data:

In [ ]:
# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# Transform data to monthly format (run or load data):
paths = {
    'csv_path': cfg.dataPath + path_PMB_GLAMOS_csv,
    'era5_climate_data':
    cfg.dataPath + path_ERA5_raw + 'era5_monthly_averaged_data.nc',
    'geopotential_data':
    cfg.dataPath + path_ERA5_raw + 'era5_geopotential_pressure.nc',
    'radiation_save_path': cfg.dataPath + path_pcsr + 'zarr/'
}
RUN = False
dataloader_gl = process_or_load_data(
    run_flag=RUN,
    data_glamos=data_glamos,
    paths=paths,
    cfg=cfg,
    vois_climate=vois_climate,
    vois_topographical=vois_topographical,
    output_file='CH_wgms_dataset_monthly_NN.csv')
data_monthly = dataloader_gl.data

data_monthly['GLWD_ID'] = data_monthly.apply(
    lambda x: mbm.data_processing.utils.get_hash(f"{x.GLACIER}_{x.YEAR}"),
    axis=1)
data_monthly['GLWD_ID'] = data_monthly['GLWD_ID'].astype(str)

dataloader_gl = mbm.dataloader.DataLoader(cfg,
                                          data=data_monthly,
                                          random_seed=cfg.seed,
                                          meta_data_columns=cfg.metaData)

In [ ]:
splits, test_set, train_set = get_CV_splits(dataloader_gl,
                                            test_split_on='GLACIER',
                                            test_splits=test_glaciers,
                                            random_state=cfg.seed)

print('Test glaciers: ({}) {}'.format(len(test_set['splits_vals']),
                                      test_set['splits_vals']))
test_perc = (len(test_set['df_X']) / len(train_set['df_X'])) * 100
print('Percentage of test size: {:.2f}%'.format(test_perc))
print('Size of test set:', len(test_set['df_X']))
print('Train glaciers: ({}) {}'.format(len(train_set['splits_vals']),
                                       train_set['splits_vals']))
print('Size of train set:', len(train_set['df_X']))

In [ ]:
# Validation and train split:
data_train = train_set['df_X']
data_train['y'] = train_set['y']
dataloader = mbm.dataloader.DataLoader(cfg, data=data_train)

train_itr, val_itr = dataloader.set_train_test_split(test_size=0.2)

# Get all indices of the training and valing dataset at once from the iterators. Once called, the iterators are empty.
train_indices, val_indices = list(train_itr), list(val_itr)

df_X_train = data_train.iloc[train_indices]
y_train = df_X_train['POINT_BALANCE'].values

# Get val set
df_X_val = data_train.iloc[val_indices]
y_val = df_X_val['POINT_BALANCE'].values

In [ ]:
features_topo = [
    'ELEVATION_DIFFERENCE',
    'pcsr',
] + list(vois_topographical)

feature_columns = features_topo + list(vois_climate)

cfg.setFeatures(feature_columns)

all_columns = feature_columns + cfg.fieldsNotFeatures

# Because CH has some extra columns, we need to cut those
df_X_train_subset = df_X_train[all_columns]
df_X_val_subset = df_X_val[all_columns]
df_X_test_subset = test_set['df_X'][all_columns]

print('Shape of training dataset:', df_X_train_subset.shape)
print('Shape of validation dataset:', df_X_val_subset.shape)
print('Shape of testing dataset:', df_X_test_subset.shape)
print('Running with features:', feature_columns)

assert all(train_set['df_X'].POINT_BALANCE == train_set['y'])

## Models:
### XGBoost model:

In [ ]:
param_init_xgb = {
    # 'device': 'cuda:0',
    'device': 'cpu',
    'tree_method': 'hist',
    "random_state": cfg.seed,
    "n_jobs": cfg.numJobs
}

custom_params = {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 800}

params = {**param_init_xgb, **custom_params}
print(params)
custom_xgb_model = mbm.models.CustomXGBoostRegressor(cfg, **params)

# Fit on train data:
custom_xgb_model.fit(df_X_train_subset, y_train)

In [ ]:
# Make predictions on test
custom_xgb_model = custom_xgb_model.set_params(device='cpu')
features_test, metadata_test = custom_xgb_model._create_features_metadata(
    test_set['df_X'][all_columns])
y_pred = custom_xgb_model.predict(features_test)
print('Shape of the test:', features_test.shape)

# Make predictions aggr to meas ID:
y_pred_agg = custom_xgb_model.aggrPredict(metadata_test, features_test)

# Calculate scores
score = custom_xgb_model.score(test_set['df_X'][all_columns],
                               test_set['y'])  # negative
print('Overall score:', np.abs(score))

grouped_ids_xgb = getDfAggregatePred(test_set, y_pred_agg, all_columns)
PlotPredictions(grouped_ids_xgb, y_pred, metadata_test, test_set,
                custom_xgb_model)
plt.suptitle(f'MBM tested on {test_glaciers}', fontsize=20)
plt.tight_layout()

In [ ]:
df_pred_xgb = pd.DataFrame(metadata_test, columns=cfg.metaData)
df_pred_xgb['pred'] = y_pred

pred_per_id = pd.DataFrame(df_pred_xgb.groupby('ID').pred.unique())
pred_per_id['MONTHS'] = df_pred_xgb.groupby('ID').MONTHS.unique()
pred_per_id.reset_index(inplace=True)

# df_pred_months_annual = df_pred_months[df_pred_months['PERIOD'] == 'annual']
months_extended = [
    'sep', 'oct', 'nov', 'dec', 'jan', 'feb', 'mar', 'apr', 'may', 'jun',
    'jul', 'aug', 'sep_', 'oct_'
]

df_months_xgb = pd.DataFrame(columns=months_extended)

for i, row in pred_per_id.iterrows():
    dic = {}
    for i, month in enumerate(row.MONTHS):
        if month in dic.keys():
            month = month + '_'
        dic[month] = row.pred[i]

    # add missing months from months extended
    for month in months_extended:
        if month not in dic.keys():
            dic[month] = np.nan

    df_months_xgb = pd.concat(
        [df_months_xgb, pd.DataFrame([dic])], ignore_index=True)
df_months_xgb = df_months_xgb.dropna(axis=1, how='all')

### NN model:

#### Initialize model:

In [ ]:
# Load model and set to CPU
model_filename = "nn_model_2025-06-06.pt"  # Replace with actual date if needed

features_topo = [
    'ELEVATION_DIFFERENCE',
    'pcsr',
] + list(vois_topographical)

feature_columns = features_topo + list(vois_climate)

cfg.setFeatures(feature_columns)

all_columns = feature_columns + cfg.fieldsNotFeatures

early_stop = EarlyStopping(
    monitor='valid_loss',
    patience=10,
    threshold=1e-4,  # Optional: stop only when improvement is very small
)

lr_scheduler_cb = LRScheduler(policy=ReduceLROnPlateau,
                              monitor='valid_loss',
                              mode='min',
                              factor=0.5,
                              patience=5,
                              threshold=0.01,
                              threshold_mode='rel',
                              verbose=True)

custom_params = {
    'lr': 0.001,
    'batch_size': 128,
    'module__layer0': 64,
    'module__layer1': 64,
    'module__layer2': 32,
    'module__layer3': 32,
    'module__dropout': 0.2,
    'optimizer': torch.optim.Adam
}

params = custom_params
param_init_NN = {'device': 'cpu'}  # Use CPU for training
nInp = len(feature_columns)

dataset = dataset_val = None  # Initialized hereafter


def my_train_split(ds, y=None, **fit_params):
    return dataset, dataset_val


args_NN = {
    'module': BiggerNetwork,
    'nbFeatures': nInp,
    'module__input_dim': nInp,
    'module__dropout': params['module__dropout'],
    'module__layer0': params['module__layer0'],
    'module__layer1': params['module__layer1'],
    'module__layer2': params['module__layer2'],
    'module__layer3': params['module__layer3'],
    'train_split': my_train_split,
    'batch_size': params['batch_size'],
    'verbose': 1,
    'iterator_train__shuffle': True,
    'lr': params['lr'],
    'max_epochs': 300,
    'optimizer': params['optimizer'],
    'callbacks': [
        ('early_stop', early_stop),
        ('lr_scheduler', lr_scheduler_cb),
    ]
}

custom_NN_model = mbm.models.CustomNeuralNetRegressor.load_model(
    cfg,
    model_filename,
    **{
        **args_NN,
        **param_init_NN
    },
)
custom_NN_model = custom_NN_model.set_params(device='cpu')
custom_NN_model = custom_NN_model.to('cpu')

In [ ]:
# Create features and metadata
features_test, metadata_test = custom_NN_model._create_features_metadata(
    df_X_test_subset)

# Ensure all tensors are on CPU if they are torch tensors
if hasattr(features_test, 'cpu'):
    features_test = features_test.cpu()

# Ensure targets are also on CPU
targets_test = test_set['y']
if hasattr(targets_test, 'cpu'):
    targets_test = targets_test.cpu()

# Create the dataset
dataset_test = mbm.data_processing.AggregatedDataset(cfg,
                                                     features=features_test,
                                                     metadata=metadata_test,
                                                     targets=targets_test)

dataset_test = [
    SliceDataset(dataset_test, idx=0),
    SliceDataset(dataset_test, idx=1)
]

# Make predictions aggr to meas ID
y_pred = custom_NN_model.predict(dataset_test[0])
y_pred_agg = custom_NN_model.aggrPredict(dataset_test[0])

batchIndex = np.arange(len(y_pred_agg))
y_true = np.array([e for e in dataset_test[1][batchIndex]])

# Calculate scores
score = custom_NN_model.score(dataset_test[0], dataset_test[1])
mse, rmse, mae, pearson = custom_NN_model.evalMetrics(y_pred, y_true)

# Aggregate predictions
id = dataset_test[0].dataset.indexToId(batchIndex)
data = {
    'target': [e[0] for e in dataset_test[1]],
    'ID': id,
    'pred': y_pred_agg
}
grouped_ids_NN = pd.DataFrame(data)

# Add period
periods_per_ids = df_X_test_subset.groupby('ID')['PERIOD'].first()
grouped_ids_NN = grouped_ids_NN.merge(periods_per_ids, on='ID')

# Add glacier name
glacier_per_ids = df_X_test_subset.groupby('ID')['GLACIER'].first()
grouped_ids_NN = grouped_ids_NN.merge(glacier_per_ids, on='ID')

# Add YEAR
years_per_ids = df_X_test_subset.groupby('ID')['YEAR'].first()
grouped_ids_NN = grouped_ids_NN.merge(years_per_ids, on='ID')

# Add MONTHS
months_per_id = test_set['df_X'][all_columns].groupby('ID')['MONTHS'].unique()
grouped_ids_NN = grouped_ids_NN.merge(months_per_id, on='ID')

grouped_ids_NN.head(3)

In [ ]:
PlotPredictions_NN(grouped_ids_NN)

## Compare models:

### Monthly distributions:

In [ ]:
df_pred_months = pd.DataFrame(y_pred)
df_pred_months['ID'] = id
df_pred_months['MONTHS'] = grouped_ids_NN['MONTHS']
df_pred_months['PERIOD'] = grouped_ids_NN['PERIOD']

months_extended = [
    'sep', 'oct', 'nov', 'dec', 'jan', 'feb', 'mar', 'apr', 'may', 'jun',
    'jul', 'aug', 'sep_', 'oct_'
]

df_months_nn = pd.DataFrame(columns=months_extended)

for i, row in df_pred_months.iterrows():
    dic = {}
    for i, month in enumerate(row.MONTHS):
        if month in dic.keys():
            month = month + '_'
        dic[month] = row[i]

    # add missing months from months extended
    for month in months_extended:
        if month not in dic.keys():
            dic[month] = np.nan
    df_months_nn = pd.concat([df_months_nn, pd.DataFrame([dic])],
                             ignore_index=True)

df_months_nn = df_months_nn.dropna(axis=1, how='all')

In [ ]:
array_nn, array_xgb, months = [], [], []

month_order = [
    'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct',
    'nov', 'dec'
]
cat_month = CategoricalDtype(month_order, ordered=True)

df_months_xgb = df_months_xgb[month_order]
df_months_nn = df_months_nn[month_order]

for col in df_months_nn.columns:
    array_nn.append(df_months_nn[col].values)
    array_xgb.append(df_months_xgb[col].values)
    months.append(np.tile(col, len(df_months_nn[col])))

df_months_nn_long = pd.DataFrame(
    data={
        'mb_nn': np.concatenate(np.array(array_nn)),
        'mb_xgb': np.concatenate(np.array(array_xgb)),
        'Month': np.concatenate(np.array(months))
    })

# order df_months_nn_long
df_months_nn_long['Month'] = df_months_nn_long['Month'].astype(cat_month)

model_colors = [color_xgb, color_tim]
alpha = 1

cm = 1 / 2.54
ax, fig = joypy.joyplot(df_months_nn_long,
                        by='Month',
                        column=['mb_xgb', 'mb_nn'],
                        alpha=0.8,
                        overlap=0,
                        fill=False,
                        linewidth=1.5,
                        xlabelsize=8.5,
                        ylabelsize=8.5,
                        x_range=[-2.2, 2.2],
                        grid=False,
                        color=model_colors,
                        figsize=(12 * cm, 14 * cm),
                        ylim='own')

vline_alpha = 0.5
plt.axvline(x=0, color='grey', alpha=vline_alpha, linewidth=1)

plt.xlabel('Mass balance (m w.e.)', fontsize=8.5)
plt.yticks(ticks=range(1, 13), labels=month_order, fontsize=8.5)
plt.gca().set_yticklabels(month_order)

legend_patches = [
    Patch(facecolor=color, label=model, alpha=alpha, edgecolor='k')
    for model, color in zip(
        ['XGB', 'NN'], model_colors)
]
plt.legend(handles=legend_patches,
           loc='upper center',
           bbox_to_anchor=(0.48, -0.1),
           ncol=4,
           fontsize=8.5,
           handletextpad=0.5,
           columnspacing=1)

### Scatter on test glaciers:

In [ ]:
fig = plt.figure(figsize=(15, 5))

grouped_ids_annual_NN = grouped_ids_NN[grouped_ids_NN.PERIOD == 'annual']
y_true_mean_NN = grouped_ids_annual_NN['target']
y_pred_agg_NN = grouped_ids_annual_NN['pred']
scores_annual_NN = {
    'mse': mean_squared_error(y_true_mean_NN, y_pred_agg_NN),
    'rmse': root_mean_squared_error(y_true_mean_NN, y_pred_agg_NN),
    'mae': mean_absolute_error(y_true_mean_NN, y_pred_agg_NN),
    'pearson_corr': np.corrcoef(y_true_mean_NN, y_pred_agg_NN)[0, 1]
}

grouped_ids_annual_xgb = grouped_ids_xgb[grouped_ids_xgb.PERIOD == 'annual']
y_true_mean_xgb = grouped_ids_annual_xgb['target']
y_pred_agg_xgb = grouped_ids_annual_xgb['pred']
scores_annual_xgb = {
    'mse': mean_squared_error(y_true_mean_xgb, y_pred_agg_xgb),
    'rmse': root_mean_squared_error(y_true_mean_xgb, y_pred_agg_xgb),
    'mae': mean_absolute_error(y_true_mean_xgb, y_pred_agg_xgb),
    'pearson_corr': np.corrcoef(y_true_mean_xgb, y_pred_agg_xgb)[0, 1]
}

# Winter
grouped_ids_winter_NN = grouped_ids_NN[grouped_ids_NN.PERIOD == 'winter']
y_true_mean_NN_w = grouped_ids_winter_NN['target']
y_pred_agg_NN_w = grouped_ids_winter_NN['pred']
scores_winter_NN = {
    'mse': mean_squared_error(y_true_mean_NN_w, y_pred_agg_NN_w),
    'rmse': root_mean_squared_error(y_true_mean_NN_w, y_pred_agg_NN_w),
    'mae': mean_absolute_error(y_true_mean_NN_w, y_pred_agg_NN_w),
    'pearson_corr': np.corrcoef(y_true_mean_NN_w, y_pred_agg_NN_w)[0, 1]
}

grouped_ids_winter_xgb = grouped_ids_xgb[grouped_ids_xgb.PERIOD == 'winter']
y_true_mean_xgb_w = grouped_ids_winter_xgb['target']
y_pred_agg_xgb_w = grouped_ids_winter_xgb['pred']
scores_winter_xgb = {
    'mse': mean_squared_error(y_true_mean_xgb_w, y_pred_agg_xgb_w),
    'rmse': root_mean_squared_error(y_true_mean_xgb_w, y_pred_agg_xgb_w),
    'mae': mean_absolute_error(y_true_mean_xgb_w, y_pred_agg_xgb_w),
    'pearson_corr': np.corrcoef(y_true_mean_xgb_w, y_pred_agg_xgb_w)[0, 1]
}

ax1 = plt.subplot(1, 2, 1)
ax1.set_title('XGB predictions', fontsize=20)
predVSTruth(ax1,
            grouped_ids_xgb,
            scores_annual_xgb,
            hue='PERIOD',
            add_legend=False,
            palette=[color_dark_blue, color_pink])

legend_xgb = "\n".join(
    ((r"$\mathrm{RMSE_a}=%.3f$, $\mathrm{RMSE_w}=%.3f$," %
      (scores_annual_xgb["rmse"], scores_winter_xgb["rmse"])),
     (r"$\mathrm{\rho_a}=%.3f$, $\mathrm{\rho_w}=%.3f$" %
      (scores_annual_xgb["pearson_corr"], scores_winter_xgb["pearson_corr"]))))
ax1.text(0.03,
         0.98,
         legend_xgb,
         transform=ax1.transAxes,
         verticalalignment="top",
         fontsize=20,
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))

ax2 = plt.subplot(1, 2, 2)
ax2.set_title('NN predictions', fontsize=20)
predVSTruth(ax2,
            grouped_ids_NN,
            scores_annual_NN,
            hue='PERIOD',
            add_legend=False,
            palette=[color_dark_blue, color_pink])

legend_NN = "\n".join(
    ((r"$\mathrm{RMSE_a}=%.3f$, $\mathrm{RMSE_w}=%.3f$," %
      (scores_annual_NN["rmse"], scores_winter_NN["rmse"])),
     (r"$\mathrm{\rho_a}=%.3f$, $\mathrm{\rho_w}=%.3f$" %
      (scores_annual_NN["pearson_corr"], scores_winter_NN["pearson_corr"]))))
ax2.text(0.03,
         0.98,
         legend_NN,
         transform=ax2.transAxes,
         verticalalignment="top",
         fontsize=20,
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))

plt.tight_layout()

### Geodetic MB:

In [ ]:
PATH_PREDICTIONS_NN = cfg.dataPath + path_distributed_MB_glamos + 'MBM/glamos_dems_NN/'
PATH_PREDICTIONS_XGB = cfg.dataPath + path_distributed_MB_glamos + 'MBM/glamos_dems/'

In [ ]:
glaciers_in_glamos = os.listdir(PATH_PREDICTIONS_NN)

geodetic_mb = get_geodetic_MB(cfg)

# get years per glacier
years_start_per_gl = geodetic_mb.groupby(
    'glacier_name')['Astart'].unique().apply(list).to_dict()
years_end_per_gl = geodetic_mb.groupby('glacier_name')['Aend'].unique().apply(
    list).to_dict()

periods_per_glacier, geoMB_per_glacier = build_periods_per_glacier(geodetic_mb)

# Glaciers with geodetic MB data:
# Sort glaciers by area
gl_area = get_gl_area(cfg)
gl_area['clariden'] = gl_area['claridenL']


# Sort the lists by area if available in gl_area
def sort_by_area(glacier_list, gl_area):
    return sorted(glacier_list, key=lambda g: gl_area.get(g, 0), reverse=False)


glacier_list = [
    f for f in list(periods_per_glacier.keys()) if f in glaciers_in_glamos
]
glacier_list = sort_by_area(glacier_list, gl_area)
# print len and list
print('Number of glaciers:', len(glacier_list))
print('Glaciers:', glacier_list)

In [ ]:
df_all_xgb = process_geodetic_mass_balance_comparison(
    glacier_list=glacier_list,
    path_SMB_GLAMOS_csv=cfg.dataPath + path_SMB_GLAMOS_csv,
    periods_per_glacier=periods_per_glacier,
    geoMB_per_glacier=geoMB_per_glacier,
    gl_area=gl_area,
    test_glaciers=test_glaciers,
    path_predictions=PATH_PREDICTIONS_XGB,  # or another path if needed
    cfg=cfg)

df_all_nn = process_geodetic_mass_balance_comparison(
    glacier_list=glacier_list,
    path_SMB_GLAMOS_csv=cfg.dataPath + path_SMB_GLAMOS_csv,
    periods_per_glacier=periods_per_glacier,
    geoMB_per_glacier=geoMB_per_glacier,
    gl_area=gl_area,
    test_glaciers=test_glaciers,
    path_predictions=PATH_PREDICTIONS_NN,  # or another path if needed
    cfg=cfg)


In [ ]:
# Drop rows where any required columns are NaN
df_all_nn = df_all_nn.dropna(subset=['Geodetic MB', 'MBM MB', 'GLAMOS MB'])

# Compute RMSE and Pearson correlation
rmse_nn = mean_squared_error(df_all_nn["Geodetic MB"],
                             df_all_nn["MBM MB"],
                             squared=False)
corr_nn = np.corrcoef(df_all_nn["Geodetic MB"], df_all_nn["MBM MB"])[0, 1]

# Drop rows where any required columns are NaN
df_all_xgb = df_all_xgb.dropna(subset=['Geodetic MB', 'MBM MB', 'GLAMOS MB'])

# Compute RMSE and Pearson correlation
rmse_xgb = mean_squared_error(df_all_xgb["Geodetic MB"],
                              df_all_xgb["MBM MB"],
                              squared=False)
corr_xgb = np.corrcoef(df_all_xgb["Geodetic MB"], df_all_xgb["MBM MB"])[0, 1]

# Define figure and axes
fig, axs = plt.subplots(1, 2, figsize=(10, 8), sharex=True, sharey = True)

# Plot MBM MB vs Geodetic MB
plot_scatter(df_all_xgb, 'GLACIER', True, axs[0], "MBM MB", rmse_xgb, corr_xgb)

axs[0].set_title('XGB predictions', fontsize=20)
axs[0].set_ylabel('Predicted mass balance [m w.e.]', fontsize=18)
axs[0].set_xlabel('Geodetic mass balance [m w.e.]', fontsize=18)

plot_scatter(df_all_nn, 'GLACIER', True, axs[1], "MBM MB", rmse_nn, corr_nn)
axs[1].set_ylabel('Predicted mass balance [m w.e.]', fontsize=18)
axs[1].set_xlabel('Geodetic mass balance [m w.e.]', fontsize=18)
axs[1].set_title('NN predictions', fontsize=20)

# Adjust legend outside of plot
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles,
           labels,
           bbox_to_anchor=(1.05, 1),
           loc="upper left",
           borderaxespad=0.,
           ncol=2,
           fontsize=20)

plt.tight_layout()
plt.show()

### Look at maps:

In [ ]:
# Load stake data ONCE instead of for every glacier
stake_file = os.path.join(cfg.dataPath, path_PMB_GLAMOS_csv,
                          "CH_wgms_dataset_all.csv")
df_stakes = pd.read_csv(stake_file)

In [ ]:
# Example usage
GLACIER_NAME = 'gietro'
df_xgb = df_all_xgb[df_all_xgb.GLACIER == GLACIER_NAME]
df_nn = df_all_nn[df_all_nn.GLACIER == GLACIER_NAME]

fig, axs = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)

plot_scatter_comparison(axs[0],
                        df_xgb,
                        GLACIER_NAME,
                        color_mbm=color_xgb,
                        color_glamos=color_tim,
                        title_suffix="(XGB)")
plot_scatter_comparison(axs[1],
                        df_nn,
                        GLACIER_NAME,
                        color_mbm=color_xgb,
                        color_glamos=color_tim,
                        title_suffix="(NN)")

plt.tight_layout()
plt.show()

In [ ]:
# Load GLAMOS data
GLAMOS_glwmb = get_GLAMOS_glwmb(GLACIER_NAME, cfg)

MBM_glwmb_nn = mbm_glwd_pred(PATH_PREDICTIONS_NN, GLACIER_NAME)
MBM_glwmb_nn.rename(columns={"MBM Balance": "MBM Balance NN"}, inplace=True)
MBM_glwmb_xgb = mbm_glwd_pred(PATH_PREDICTIONS_XGB, GLACIER_NAME)
MBM_glwmb_xgb.rename(columns={"MBM Balance": "MBM Balance XGB"}, inplace=True)

# Merge with GLAMOS data
MBM_glwmb_nn = MBM_glwmb_nn.join(GLAMOS_glwmb)

# Drop NaN values to avoid plotting errors
MBM_glwmb_nn = MBM_glwmb_nn.dropna()

MBM_glwmb = MBM_glwmb_nn.join(MBM_glwmb_xgb)
# Plot the data
fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharey=True)
MBM_glwmb.plot(ax=axs[0],
               y=['MBM Balance XGB', 'GLAMOS Balance'],
               marker="o",
               color=[color_xgb, color_tim])
MBM_glwmb.plot(ax=axs[1],
               y=['MBM Balance NN', 'GLAMOS Balance'],
               marker="o",
               color=[color_xgb, color_tim])

for ax in axs:
    ax.set_title(f"{GLACIER_NAME.capitalize()} Glacier", fontsize=24)
    ax.set_ylabel("Mass Balance [m w.e.]", fontsize=18)
    ax.set_xlabel("Year", fontsize=18)
    ax.grid(True, linestyle="--", linewidth=0.5)
    ax.legend(fontsize=14)

axs[0].set_title(f"{GLACIER_NAME.capitalize()} Glacier (XGB)", fontsize=16)
axs[1].set_title(f"{GLACIER_NAME.capitalize()} Glacier (NN)", fontsize=16)

plt.tight_layout()
plt.show()

In [ ]:
for year in MBM_glwmb_nn.index:
    plot_mass_balance_comparison_annual(
        glacier_name=GLACIER_NAME,
        year=year,
        cfg=cfg,
        df_stakes=df_stakes,
        path_distributed_mb=path_distributed_MB_glamos,
        path_pred_xgb=PATH_PREDICTIONS_XGB,
        path_pred_nn=PATH_PREDICTIONS_NN,
        get_glamos_func=get_GLAMOS_glwmb,
        get_pred_func=get_predicted_mb,
        get_glamos_pred_func=get_predicted_mb_glamos,
        load_grid_func=load_grid_file,
        to_wgs84_func=transform_xarray_coords_lv95_to_wgs84,
        apply_filter_func=apply_gaussian_filter,
        get_colormaps_func=get_color_maps)
